# Klasy

In [ ]:
class Gadget:
    def __init__(self, id: int, name: str) -> None:
        self.id = id
        self.name = name

    def use(self) -> None:
        print(f"Using {self.name} with Id#{self.id}")

    def __repr__(self) -> str:
        return f"Gadget(id={self.id}, name={self.name!r})"

    def __str__(self) -> str:
        return f"Gadget - id: {self.id}; name: {self.name}"
    
    def __eq__(self, other):
        return (self.id, self.name) == (other.id, other.name)

In [ ]:
(5).__add__(7)

In [ ]:
g_1 = Gadget(1, "ipad")

In [ ]:
type(g_1)

In [ ]:
id(g_1)

In [ ]:
g_1.name = "ipad 2.0"

In [ ]:
g_1.use()

In [ ]:
print(g_1)

In [ ]:
g_1

In [ ]:
g_2 = eval(repr(g_1))

In [ ]:
g_2

In [ ]:
g_1 == g_2

In [ ]:
g_1.__eq__(g_2)

# Metody `__new__` & `__init__`

In [ ]:
from typing import Self


class Dummy:
    def __new__(cls, *args) -> Self:
        print(f"Dummy.__new__({cls}, {args}) has been called...")
        obj = super().__new__(cls)
        obj.extra_attribute = "EXTRA"
        print(f"Object {obj} has been created...")
        return obj

    def __init__(self, *args) -> None:
        print(f"Dummy.__init__({self}, {args})...")
        self.args = args
        print(f"Object's __dict__: { self.__dict__}")

In [ ]:
d = Dummy(1, "one")

## Kiedy używać `__new__`?

In [ ]:
class UppercaseTuple(tuple):
    def __init__(self, list) -> None:
        print(f"Start changes for {list}")

        for i, item in enumerate(list):
            self[i] = item.upper()

In [ ]:
UppercaseTuple(['one', 'two', 'three'])

In [ ]:
class UppercaseTuple(tuple):
    def __new__(cls, list):
        print(f"Start changes for {list}")
        new_content = [item.upper() for item in list]       
        return super().__new__(cls, new_content)

In [ ]:
UppercaseTuple(['one', 'two', 'three'])